In [1]:
import yaml

def base_yaml_all(base_file):
  with open(base_file) as f:
    bases = list(yaml.safe_load_all(f))
  return bases

def base_yaml(base_file):
  with open(base_file) as f:
    base = yaml.safe_load(f)
  return base

In [2]:
base_dir = "base-yamls/"
ns_base = base_yaml(base_dir + "namespace.yaml")
fortio_server = base_yaml(base_dir + "fortio-server-dp.yaml")
iperf3_server = base_yaml(base_dir + "iperf3-server-dp.yaml")
iperf3_svc = base_yaml(base_dir + "iperf3-server-svc.yaml")
fortio_svc = base_yaml(base_dir + "fortio-server-svc.yaml")
dst_rule = base_yaml(base_dir + "destination-rule.yaml")
svs_monitor = base_yaml(base_dir + "svc-monitor.yaml")
client_fortio = base_yaml_all(base_dir + "client-fortio-job.yaml")
client_iperf3 = base_yaml_all(base_dir + "client-iperf3-job.yaml")
pv = base_yaml(base_dir + "pv.yaml")
pvc = base_yaml(base_dir + "pvc.yaml")

In [3]:
# AWS EAST
l = dict()
yamls = {}
namespace = "loadtest"

l["aws_east"] = {
  "labels": {
    "provider": "aws-us-east-1"
  }
}
l["aws_west"] = {
  "labels": {
    "provider": "aws-us-west-1"
  }
}

yamls["aws_east"] = []
yamls["aws_west"] = []

In [4]:
# case1 prepration: only one cluster has servers,
# and services uses LoadBalancer (direct communication)

# For all clusters
for k, v in l.items():
  d = ns_base.copy()
  d["metadata"]["name"] = namespace
  yamls[k].append(d)

  d = fortio_svc.copy()
  d["metadata"]["namespace"] = namespace
  d["spec"]["type"] = "LoadBalancer"
  yamls[k].append(d)

  d = iperf3_svc.copy()
  d["metadata"]["namespace"] = namespace
  d["spec"]["type"] = "LoadBalancer"
  yamls[k].append(d)

  d = dst_rule.copy()
  d["metadata"]["namespace"] = namespace
  # We want to prioritize aws east (i.e. client in west goes to east first)
  d["spec"]["trafficPolicy"]["loadBalancer"]["localityLbSetting"]["failoverPriority"] = ["provider=aws-us-east-1"]
  yamls[k].append(d)

  # no service monitors for this app. 

k = "aws_east"
v = l[k]

d = fortio_server.copy()
d["metadata"]["labels"] = v["labels"]
d["metadata"]["namespace"] = namespace
yamls[k].append(d)

d = iperf3_server.copy()
d["metadata"]["labels"] = v["labels"]
d["metadata"]["namespace"] = namespace
yamls[k].append(d)

for k, v in yamls.items():
  print(f"# YAMLs for cluster: {k}")
  with open(f"deploy-{k}.yaml", "w") as f:
    out = "---\n".join([yaml.dump(doc) for doc in v])
    f.write(out)
  

# YAMLs for cluster: aws_east
# YAMLs for cluster: aws_west


In [5]:
# Once deployed, get the external service IP and use it

fortio_ext_address = "a7539eda8f1724befb3e21c123acb3e6-1295078554.us-east-1.elb.amazonaws.com"
iperf3_ext_address = "ab9ae01336c0e4acc861e533b2de070c-1901118216.us-east-1.elb.amazonaws.com"

yamls_west = []
k = "aws_west"
v = l[k]
d = client_fortio[0].copy()
d["metadata"]["namespace"] = namespace
d["spec"]["template"]["spec"]["containers"][0]["env"][0]["value"] = fortio_ext_address
yamls_west.append(d)

d = client_iperf3[0].copy()
d["metadata"]["namespace"] = namespace
d["spec"]["template"]["spec"]["containers"][0]["env"][0]["value"] = iperf3_ext_address
yamls_west.append(d)

# add PV and PVC to store results
d = pv.copy()
yamls_west.append(d)

d = pvc.copy()
d["metadata"]["namespace"] = namespace
yamls_west.append(d)

with open(f"deploy-{k}-clients.yaml", "w") as f:
  out = "---\n".join([yaml.dump(doc) for doc in yamls_west])
  f.write(out)
